In [1]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML
import pandas as pd

/tmp/ipykernel_84545/241899341.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import numpy as np
from scipy.integrate import odeint
from tqdm import tqdm
import time
import random

# Constants
a21 = 1
a31 = 0.27
a32 = 0.73

a41 = 0.18
a42 = 0.24
a43 = 0.58

a51 = 0.24
a52 = 0.23
a53 = 0.2
a54 = 0.33

W2 = 63000
W3 = 20000
W4 = 15000
W5 = 33000
Ws2 = 8000

P980 = 3*10**4  # our PD



def system(state, t, x, c1, c2, c3, c4, k31, k41, k51):

    ns2, n1, n2, n3, n4, n5 = state

    #ns2
    ms2 = 1.23*P980*(37*(100-x)-ns2) - Ws2*ns2 - (c1*n1+c2*n2+c3*n3+c4*n4)*ns2  # ns1 = total_Yb - ns2

    # n1
    m1 = -c1*n1*ns2 + a21*W2*n2 + a31*W3*n3 + a41*W4*n4 + a51*W5*n5 - k41*n1*n4 - k31*n1*n3 - k51*n5*n1

    # n2
    m2 = c1*n1*ns2 - c2*n2*ns2 - a21*W2*n2 + a32*W3*n3 + a42*W4*n4 + a52*W5*n5 + k41*n1*n4 + 2*k31*n1*n3

    # n3
    m3 = c2*n2*ns2 - c3*n3*ns2 - (a31+a32)*W3*n3 + a43*W4*n4 + a53*W5*n5 + 2*k51*n1*n5 + k41*n1*n4 - k31*n1*n3

    # n4
    m4 = c3*n3*ns2 - c4*n4*ns2 - (a43+a42+a41)*W4*n4 + a54*W5*n5 - k41*n1*n4

    # n5
    m5 = c4*n4*ns2 - (a54+a53+a52+a51)*W5*n5 - k51*n1*n5


    return [ms2, m1, m2, m3, m4, m5]

In [4]:
syn = pd.read_csv('ga_syn_1.csv')

In [50]:
from scipy.integrate import odeint
from tqdm import tqdm

def compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51):

    # Compute initial condition
    y = 37 * x
    state0 = [0, y, 0, 0, 0, 0]

    # ODEs
    t = np.arange(0.0, 0.001, 0.000001)

    state = odeint(system, state0, t, args=(x, c1, c2, c3, c4, k31, k41, k51))

    # Compute NIR and blue_total
    NIR = a31 * W3 * state[:, 3][-1]
    blue_1 = a41 * W4 * state[:, 4][-1]
    blue_2 = a52 * W5 * state[:, 5][-1]
    blue_total = blue_1 + blue_2

    return NIR, blue_total


x_values = [4, 6, 8, 10, 12, 15, 50]  # Range of C2 values from 2% to 40%


In [13]:
Cr = 8 / 100

def calculate_up(up, C2):
    C2 = C2 / 100
    return 3 * up / (2 + (Cr / C2) ** 2)


def calculate_k(k, C2):
    C2 = C2 / 100
    return k * (C2 / Cr) ** 2

In [58]:
NIR_values = {x: [] for x in x_values}
blue_total_values = {x: [] for x in x_values}

count = 0
for index, row in syn.iterrows():
    c1, c2, c3, c4, k31, k41, k51 = row['c1'], row['c2'], row['c3'],row['c4'],row['k31'],row['k41'],row['k51']
    c1, c2, c3, c4, k31, k41, k51 = 10**c1, 10**c2, 10**c3, 10**c4, 10**k31, 10**k41, 10**k51
    up_values = [c1, c2, c3, c4]
    k_values = [k31, k41, k51]
    C2_range = [4, 6, 8, 10, 12, 15, 50]
    nested_up_list = []
    nested_k_list = []
    for C2 in C2_range:
        up_sublist = [calculate_up(up, C2) for up in up_values]
        k_sublist = [calculate_k(k, C2) for k in k_values]
        nested_up_list.append(up_sublist)
        nested_k_list.append(k_sublist)
    print(nested_up_list, nested_k_list)
    for index, x in enumerate(x_values):
        c1, c2, c3, c4 = nested_up_list[index]
        k31, k41, k51 = nested_k_list[index]
        NIR, blue_total = compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51)
        # print('c1234, k3141501: ', c1, c2, c3, c4, k31, k41, k51)
        # print(NIR)
        # print(blue_total)
        NIR_values[x].append(NIR)
        blue_total_values[x].append(blue_total)
    print(NIR_values[4])
    print(blue_total_values[4])
    count += 1
    # if count >=100:
    #     break


[[1333.8498887328628, 0.0072478135983081, 3.2853147027185314, 1.1270140700426954e-07], [2118.467470340429, 0.011511233362018746, 5.217852763141196, 1.7899635230089866e-07], [2667.6997774657257, 0.0144956271966162, 6.570629405437063, 2.2540281400853908e-07], [3031.4770198474157, 0.01647230363251841, 7.466624324360299, 2.561395613733399e-07], [3273.9951814352085, 0.01779008792311988, 8.063954270309122, 2.7663072628320705e-07], [3503.3022369442897, 0.019036086299058237, 8.62874484177046, 2.9600564096452116e-07], [3950.977158568907, 0.021468642175083236, 9.731382413265793, 3.338311818846847e-07]] [[0.047535610423267455, 3.3565807876240943e-11, 0.0028903051172732234], [0.10695512345235178, 7.552306772154212e-11, 0.006503186513864752], [0.19014244169306982, 1.3426323150496377e-10, 0.011561220469092894], [0.29709756514542157, 2.097862992265059e-10, 0.018064406982957647], [0.4278204938094071, 3.020922708861685e-10, 0.02601274605545901], [0.6684695215771986, 4.720191732596383e-10, 0.04064491571

In [ ]:
1333.8498887328628 0.0072478135983081 3.2853147027185314 1.1270140700426954e-07 0.047535610423267455 3.3565807876241085e-11 0.0028903051172732234

In [59]:
nir_mean_values = [np.mean(NIR_values[x]) for x in x_values]
nir_std_values = [np.std(NIR_values[x]) for x in x_values]

blue_mean_values = [np.mean(blue_total_values[x]) for x in x_values]
blue_std_values = [np.std(blue_total_values[x]) for x in x_values]

In [62]:
nir_mean_values, nir_std_values

([1439.9362856402327,
  3152.649577506874,
  4092.0305064430686,
  4973.962932197278,
  5717.92810431109,
  6509.1163957769595,
  76.82792820719868],
 [1206.8711606378458,
  2556.2064579596204,
  2344.3639048583714,
  2330.5157560711095,
  2457.12744318531,
  2718.4873724567497,
  41.067009453118814])

In [63]:
blue_mean_values, blue_std_values

([267.7061146478702,
  820.2839223705295,
  1194.740335024966,
  1506.9265927207284,
  1711.641753562481,
  1823.1231434058532,
  0.43682368900943863],
 [144.3840755942063,
  440.6318750039632,
  268.73028032447246,
  142.65168178944128,
  87.49397582694732,
  140.73817503445224,
  0.36886608875153265])

In [46]:
def objective_syn(exponents):
    print('hello')

    # c1, c2, c3, c4, k31, k41, k51 = params
    c1_exp, c2_exp, c3_exp, c4_exp, k31_exp, k41_exp, k51_exp = exponents

    # Converting exponent values to actual values
    c1, c2, c3, c4, k31, k41, k51 = (10**c1_exp, 10**c2_exp, 10**c3_exp,
                                 10**c4_exp, 10**k31_exp, 10**k41_exp, 10**k51_exp)

    up_values = [c1, c2, c3, c4]
    k_values = [k31, k41, k51]

    C2_range = [4, 6, 8, 10, 12, 15, 50]

    # storing new cross relaxation and new up-conversion values
    nested_up_list = []
    nested_k_list = []

    for C2 in C2_range:

        up_sublist = [calculate_up(up, C2) for up in up_values]
        k_sublist = [calculate_k(k, C2) for k in k_values]
        nested_up_list.append(up_sublist)
        nested_k_list.append(k_sublist)

    print(nested_up_list)
    print(nested_k_list)

    x_values = [4, 6, 8, 10, 12, 15, 50]

    df_syn_data = pd.read_csv("synthetic_data.csv")    

    exp_blue_total = list(df_syn_data.iloc[1])
    # [4.5*10**3, 5.7*10**3, 5.5*10**3, 3.5*10**3, 2*10**3, 1.2*10**3, 0.6*10**3]
    # blue_std=[1*10**3, 1.5*10**3, 1*10**3, 1*10**3, 0.5*10**3, 0.5*10**3, 0.25*10**3]

    #blue_weight=[i/j for i, j in zip(exp_blue_total, blue_std)]
    #blue_weight_2=[x/sum(blue_weight) for x in blue_weight]

    exp_NIR = list(df_syn_data.iloc[0])
    # [5*10**3, 8.5*10**3, 6.6*10**3, 7.5*10**3, 6*10**3, 5.5*10**3, 0.6*10**3]
    # NIR_std=[1.1*10**3, 2.25*10**3, 0.7*10**3, 1.5*10**3, 1*10**3, 1.7*10**3, 0.25*10**3]

    #NIR_weight=[i/j for i, j in zip(exp_NIR, NIR_std)]
    #NIR_weight_2=[x/sum(NIR_weight) for x in NIR_weight]

    total_error = 0

    for index, x in enumerate(x_values):

        c1, c2, c3, c4 = nested_up_list[index]
        k31, k41, k51 = nested_k_list[index]
        print('c1234, k3141501: ', c1, c2, c3, c4, k31, k41, k51)
        NIR, blue_total = compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51)

        print(NIR, exp_NIR[index])
        print(blue_total, exp_blue_total[index])

        error_NIR = ((NIR - exp_NIR[index])/exp_NIR[index])**2
        error_blue = ((blue_total - exp_blue_total[index])/exp_blue_total[index])**2
        total_error += error_NIR + error_blue

    return total_error, # a return statement with a trailing comma, it means the function is returning a tuple with a single element.


In [47]:
x = objective_syn( [3.426136952558433,-1.8387629888542367,0.817606972941167,
              -6.647040666369984,-0.7209209334458407,-9.872042904239695,
              -1.9369963168658189])
print(x)

hello
[[1333.8498887328628, 0.0072478135983081, 3.2853147027185314, 1.1270140700426954e-07], [2118.467470340429, 0.011511233362018746, 5.217852763141196, 1.7899635230089866e-07], [2667.6997774657257, 0.0144956271966162, 6.570629405437063, 2.2540281400853908e-07], [3031.4770198474157, 0.01647230363251841, 7.466624324360299, 2.561395613733399e-07], [3273.9951814352085, 0.01779008792311988, 8.063954270309122, 2.7663072628320705e-07], [3503.3022369442897, 0.019036086299058237, 8.62874484177046, 2.9600564096452116e-07], [3950.977158568907, 0.021468642175083236, 9.731382413265793, 3.338311818846847e-07]]
[[0.047535610423267455, 3.3565807876241085e-11, 0.0028903051172732234], [0.10695512345235178, 7.552306772154244e-11, 0.006503186513864752], [0.19014244169306982, 1.3426323150496434e-10, 0.011561220469092894], [0.29709756514542157, 2.0978629922650679e-10, 0.018064406982957647], [0.4278204938094071, 3.020922708861698e-10, 0.02601274605545901], [0.6684695215771986, 4.720191732596403e-10, 0.0406

In [67]:
x_values + x_values[::-1]

[4, 6, 8, 10, 12, 15, 50, 50, 15, 12, 10, 8, 6, 4]

In [68]:
nir_mean_values + nir_std_values + nir_mean_values[::-1]

[1439.9362856402327,
 3152.649577506874,
 4092.0305064430686,
 4973.962932197278,
 5717.92810431109,
 6509.1163957769595,
 76.82792820719868,
 1206.8711606378458,
 2556.2064579596204,
 2344.3639048583714,
 2330.5157560711095,
 2457.12744318531,
 2718.4873724567497,
 41.067009453118814,
 76.82792820719868,
 6509.1163957769595,
 5717.92810431109,
 4973.962932197278,
 4092.0305064430686,
 3152.649577506874,
 1439.9362856402327]

In [77]:
fig = go.Figure()

# Add the main trace (mean curve)
fig.add_trace(go.Scatter(x=x_values, y=nir_mean_values, mode='lines+markers', name='Mean'))
fig.add_trace(go.Scatter(x=x_values, y=blue_mean_values, mode='lines+markers', name='Mean'))

# Add the uncertainty shade trace
fig.add_trace(go.Scatter(
    x=x_values + x_values[::-1],
    y=[i+j for i, j in zip(nir_mean_values,nir_std_values)] + [i-j for i, j in zip(nir_mean_values,nir_std_values)][::-1],
    fill='toself',
    fillcolor='rgba(170, 68, 153, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='NIR_simulation'
))

fig.add_trace(go.Scatter(
    x=x_values + x_values[::-1],
    y=[i+j for i, j in zip(blue_mean_values,blue_std_values)] + [i-j for i, j in zip(blue_mean_values,blue_std_values)][::-1],
    fill='toself',
    fillcolor='rgba(170, 68, 153, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='NIR_simulation'
))

# exp_blue_total = [4.5*10**3, 5.7*10**3, 5.5*10**3, 3.5*10**3, 2*10**3, 1.2*10**3, 0.6*10**3]
# exp_NIR = [5*10**3, 8.5*10**3, 6.6*10**3, 7.5*10**3, 6*10**3, 5.5*10**3, 0.6*10**3]
# exp_sum = [i+j for i, j in zip(exp_blue_total, exp_NIR)]

y_ticks = [i for i in range(int(min(min(exp_sum), min(blue_total_values)) / 1e3),
                            int(max(max(exp_sum), max(blue_total_values)) / 1e3) + 1)]

fig.update_yaxes(tickvals=[tick * 1e3 for tick in y_ticks],
                 ticktext=[f"{tick} × 10^3" for tick in y_ticks])


fig.update_xaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white', dtick=2)
fig.update_yaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white')

# ... Your existing layout and formatting ...

fig.update_layout(
    title_text='NaYbF4: (1-x)% Yb, x% Tm',  # Main Title
    title_font=dict(size=24, family="Arial, sans-serif", color='royalblue'),
    width=800, height=600,
    xaxis_title='Tm concentration(%)',
    yaxis_title='Intensity(kpps)',
    title_x=0.42,  # Center the main title
    title_y=0.95,  # Adjust the vertical position of the main title
    showlegend=True, margin=dict(l=20, r=20, b=20, t=70),
    legend=dict(
        font=dict(
            size=20,
            family="Arial, sans-serif"
        )
    )
)

syn_data = pd.read_csv('synthetic_data_1.csv')
exp_NIR = list(syn_data.iloc[0])
exp_blue_total = list(syn_data.iloc[1])
fig.add_trace(go.Scatter(x= [4, 6, 8, 10, 12, 15, 50], y= exp_NIR, mode='markers', name='NIR_exp', marker=dict(color='rgb(170, 68, 153)', size=10)))
fig.add_trace(go.Scatter(x= [4, 6, 8, 10, 12, 15, 50], y= exp_blue_total, mode='markers', name='Blue_exp', marker=dict(color='#0099C6', size=10)))

# Show the plot
fig.show()
